In [104]:
import torch
import json
import cv2
import numpy as np
import os
import torch.utils.data as data
import glob
from matplotlib import pyplot as plt
from pywget import wget

In [105]:
# link = "https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip"
# wget.download(link)

In [106]:
# !unzip balloon_dataset.zip

In [139]:
class BalloonDatasetSegmentation(data.Dataset):
    def __init__(self, folder_path):
        super(BalloonDatasetSegmentation, self).__init__()
        self.img_files = glob.glob(os.path.join(folder_path, '*.jpg'))
        self.mask_files = []
        
        for img_path in self.img_files:
            self.mask_files.append(os.path.join(folder_path, os.path.splitext(os.path.basename(img_path))[0] + '_mask.png'))
    def __getitem__(self, index):
            img_path = self.img_files[index]
            mask_path = self.mask_files[index]
            data = cv2.imread(img_path)
            label = cv2. imread(mask_path)
            return torch.from_numpy(data).float(), torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)

In [134]:
def create_labels(dataset_type='train'):
    f = open(f'balloon\\{dataset_type}\\via_region_data.json')
    json_file = json.load(f)
    for file in json_file:
        path = f'balloon\\{dataset_type}'
        filename = json_file[file]['filename']

        path_to_mask = f"{path}\\{filename.replace('.jpg', '_mask.png')}"

        if os.path.exists(path_to_mask):
            continue
        else:                
            img = cv2.imread(f'{path}\\{filename}')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


            regions = json_file[file]['regions']
            label = np.zeros_like(img)

            for n in regions:
                mask = np.zeros((img.shape[0], img.shape[1]))
                x_coords = regions[n]['shape_attributes']['all_points_x']
                y_coords = regions[n]['shape_attributes']['all_points_y']
                coords_of_polygon = np.dstack((x_coords, y_coords))

                cv2.fillConvexPoly(mask, coords_of_polygon, 1)
                mask = mask.astype(bool)
                label[mask] = 1
            cv2.imwrite(path_to_mask, label)

In [135]:
create_labels('train')
create_labels('val')

In [141]:
train_dataset = BalloonDatasetSegmentation('balloon\\train')
val_dataset = BalloonDatasetSegmentation('balloon\\val')